# HOMER DATA PROCESSING SCRIPT
The goal of this script is to take the output file of HOMER Pro Batch Runs (.xlsx file containing one optimal case/sheet) and produce a condensed data set + plots.

General flow:
1) Run HOMER study via batch process. Select "export results" and name the file.


2) Import file to this script.


3) Use Pandas to pull .xlsx data into a more usable format. 


4) Output a trimmed .csv/HDF5 that holds the data as a dataframe


Goals:


Have this script take a large HOMER study (>100 cases) and build one easily accessible database.


Potentially couple this script with a HOMER input reading script to do cartography w/HOMER geography + results.


In [55]:
# Block of dependencies and working directory
import matplotlib.pyplot as plt
import conda
import numpy  as np 
import pandas as pd
import os 
import shutil
from pathlib import Path
from cycler import cycler

loc_directory =  os.getcwd() + '/Data' #home PC
print(loc_directory)
mlr_directory =  os.getcwd() + '/MLR_data' #home PC

util_directory = os.getcwd() +  '/util_data' #home PC
#directory = 'c:/repo/homer/ClimateStudies/OutputProcessing/Data' #laptop

trim_directory = os.getcwd() + '/TrimmedData'
print(trim_directory)

d:\repo\homer\ClimateStudies\OutputProcessing/Data
d:\repo\homer\ClimateStudies\OutputProcessing/TrimmedData


## Following this guide for xlsx reading
https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook

In [56]:
# reads, maps all sheets for locational variance studies
files = os.listdir(loc_directory)
output_df = {} #creates empty dictionary
names = [] #creates empty list for future name storage
for file in files:
    if file.endswith('.xlsx'):
        xls = pd.ExcelFile(loc_directory + '/' + file) #reads file
        names = names + xls.sheet_names
        for sheet_name in xls.sheet_names: #pulls individual sheets
            output_df[sheet_name] = xls.parse(sheet_name) #puts each sheet into a dictionary w/key "sheet_name"
display(output_df[sheet_name])

,Architecture/PV\n(kW),Architecture/G1500,Architecture/GenLarge\n(kW),Architecture/1kWh LI,Cost/NPC\n($),Cost/COE\n($),Cost/Operating cost\n($/yr),Cost/Initial capital\n($),System/Ren Frac\n(%),System/Total Fuel\n(L/yr),...,GenLarge/Fuel Cost\n($/yr),PV/Capital Cost\n($),PV/Production\n(kWh/yr),G1500/Capital Cost\n($),G1500/Production\n(kWh/yr),G1500/O&M Cost\n($),1kWh LI/Autonomy\n(hr),1kWh LI/Annual Throughput\n(kWh/yr),1kWh LI/Nominal Capacity\n(kWh),1kWh LI/Usable Nominal Capacity\n(kWh)
0,74305.687555,13,NaN,77203,268865728,0.220980,4799146.50,102043144,100.000000,0,...,NaN,55729264.0,117888568,19987500,33314920.0,497250,15.440631,10027248.0,77203.156250,61762.523438
1,74469.297134,13,NaN,77165,269061952,0.221138,4801634.00,102152896,100.000000,0,...,NaN,55851972.0,118148136,19987500,33314920.0,497250,15.433031,10026532.0,77165.156250,61732.125000
2,74416.394418,14,NaN,74254,269883712,0.221812,4810742.00,102658064,100.000000,0,...,NaN,55812296.0,118064208,21525000,35877608.0,535500,14.850830,9799102.0,74254.148438,59403.320312
3,78536.714041,12,NaN,76795,270071744,0.221960,4790786.00,103539784,100.000000,0,...,NaN,58902536.0,124601232,18450000,30752234.0,459000,15.359031,10252882.0,76795.156250,61436.121094
4,80596.659039,12,NaN,74576,270331520,0.222182,4775581.50,104328064,100.000000,0,...,NaN,60447496.0,127869408,18450000,30752234.0,459000,14.915230,10237940.0,74576.148438,59660.917969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,17769.626864,7,4000.0,23816,313712672,0.257721,2919993.00,212211136,72.025818,2527368,...,0.0,13327220.0,28192132,10762500,17938804.0,267750,4.763209,6214931.0,23816.046875,19052.837891
1996,11438.409326,5,4000.0,36686,313713984,0.257658,3018841.50,208776384,69.634583,2744088,...,0.0,8578807.0,18147436,7687500,12813431.0,191250,7.337214,8329274.5,36686.074219,29348.859375
1997,15420.093331,7,4000.0,32524,313719904,0.257646,2885469.75,213418416,76.179726,2152752,...,0.0,11565070.0,24464516,10762500,17938804.0,267750,6.504813,7835992.0,32524.064453,26019.251953
1998,11869.627970,7,4000.0,36720,313722048,0.257635,2920974.50,212186400,75.016251,2258016,...,0.0,8902221.0,18831578,10762500,17938804.0,267750,7.344015,8185626.0,36720.074219,29376.058594


In [57]:
# reads, maps all sheets for MLR variance studies
mlr_files = os.listdir(mlr_directory)
mlr_output_df = {} #creates empty dictionary
mlr_names = [] #creates empty list for future name storage
for file in mlr_files:
    if file.endswith('.xlsx'):
        mlr_xls = pd.ExcelFile(mlr_directory + '/' + file) #reads file
        mlr_names = mlr_names + mlr_xls.sheet_names
        for sheet_name in mlr_xls.sheet_names: #pulls individual sheets
            mlr_output_df[sheet_name] = mlr_xls.parse(sheet_name) #puts each sheet into a dictionary w/key "sheet_name"
display(mlr_output_df[sheet_name])

,Architecture/PV\n(kW),Architecture/G1500,Architecture/GenLarge\n(kW),Architecture/1kWh LI,Cost/NPC\n($),Cost/COE\n($),Cost/Operating cost\n($/yr),Cost/Initial capital\n($),System/Ren Frac\n(%),System/Total Fuel\n(L/yr),...,GenLarge/Fuel Cost\n($/yr),PV/Capital Cost\n($),PV/Production\n(kWh/yr),G1500/Capital Cost\n($),G1500/Production\n(kWh/yr),G1500/O&M Cost\n($),1kWh LI/Autonomy\n(hr),1kWh LI/Annual Throughput\n(kWh/yr),1kWh LI/Nominal Capacity\n(kWh),1kWh LI/Usable Nominal Capacity\n(kWh)
0,18946.593167,2.0,3000.0,34290,194064576,0.159500,2677911.75,100977992,75.195755,2239956,...,0.0,14209945.0,43882384,3075000.0,3863899.250,76500.0,6.858014,8215316.0,34290.070312,27432.054688
1,17979.473079,3.0,3000.0,34373,194263808,0.159660,2659464.75,101818456,76.833389,2092122,...,0.0,13484605.0,41642428,4612500.0,5795848.500,114750.0,6.874614,8134020.5,34373.070312,27498.455078
2,19545.898656,3.0,3000.0,30526,194697344,0.159989,2675878.00,101681448,75.594788,2204352,...,0.0,14659424.0,45270440,4612500.0,5795848.500,114750.0,6.105212,7621761.0,30526.060547,24420.849609
3,18532.952472,3.0,3000.0,34185,194784480,0.160094,2664345.75,102169456,77.055397,2071998,...,0.0,13899714.0,42924344,4612500.0,5795848.500,114750.0,6.837014,8175929.5,34185.070312,27348.054688
4,19482.187333,2.0,3000.0,34791,194825936,0.160114,2683343.75,101550528,75.548996,2208222,...,0.0,14611640.0,45122876,3075000.0,3863899.250,76500.0,6.958214,8287221.5,34791.070312,27832.855469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,12649.292270,NaN,4000.0,34337,218591312,0.179522,2916935.50,117196040,64.166656,3238416,...,0.0,9486969.0,29297144,NaN,NaN,NaN,6.867414,8833302.0,34337.070312,27469.654297
1996,15479.661648,1.0,4000.0,25154,218600288,0.179579,2901979.25,117724912,61.837894,3447912,...,0.0,11609746.0,35852592,1537500.0,1931949.625,38250.0,5.030810,6712273.0,25154.050781,20123.240234
1997,11208.076068,4.0,4000.0,27077,218607488,0.179594,2842793.25,119789472,67.513206,2935008,...,0.0,8406057.0,25959130,6150000.0,7727798.500,153000.0,5.415411,7067603.5,27077.054688,21661.642578
1998,9169.771566,2.0,4000.0,25301,218608000,0.179669,2992670.50,114580128,58.471809,3750288,...,0.0,6877328.5,21238194,3075000.0,3863899.250,76500.0,5.060210,6517933.0,25301.050781,20240.839844


In [58]:
# reads, maps all sheets for utility variance studies
util_files = os.listdir(util_directory)
print(os.listdir)
util_output_df = {} #creates empty dictionary
util_names = [] #creates empty list for future name storage
for file in util_files:
    if file.endswith('.xlsx'):
        util_xls = pd.ExcelFile(util_directory + '/' + file) #reads file
        util_names = util_names + util_xls.sheet_names
        # print(util_names)
        for sheet_name in util_xls.sheet_names: #pulls individual sheets
            util_output_df[sheet_name] = util_xls.parse(sheet_name) #puts each sheet into a dictionary w/key "sheet_name"
display(util_output_df[sheet_name])

<built-in function listdir>


,Architecture/PV\n(kW),Architecture/G1500,Architecture/1kWh LI,Architecture/Grid\n(kW),Architecture/Converter\n(kW),Cost/NPC\n($),Cost/COE\n($),Cost/Operating cost\n($/yr),Cost/Initial capital\n($),System/Ren Frac\n(%),...,G1500/Production\n(kWh/yr),G1500/O&M Cost\n($),1kWh LI/Autonomy\n(hr),1kWh LI/Annual Throughput\n(kWh/yr),1kWh LI/Nominal Capacity\n(kWh),1kWh LI/Usable Nominal Capacity\n(kWh),Converter/Rectifier Mean Output\n(kW),Converter/Inverter Mean Output\n(kW),Grid/Energy Purchased\n(kWh),Grid/Energy Sold\n(kWh)
0,23972.819896,1.0,37466,2600,8384.838945,113839296,0.093565,2345679.25,32301406,64.624146,...,1931949.625,38250.0,7.493215,4615358.50,37466.074219,29972.859375,557.255859,474.838806,12382070,-3.296918e-12
1,24939.900014,1.0,36414,2600,9391.171460,113870048,0.093590,2335989.25,32668990,64.732750,...,1931949.625,38250.0,7.282815,4607711.00,36414.074219,29131.257812,556.272339,474.052002,12344103,-6.821210e-12
2,24512.001887,1.0,36891,2600,9146.789407,113870344,0.093591,2340557.75,32510480,64.685966,...,1931949.625,38250.0,7.378215,4610983.00,36891.074219,29512.859375,556.694702,474.388611,12360450,-5.684342e-12
3,24029.556936,1.0,37428,2600,8410.198819,113870976,0.093591,2345738.50,32331026,64.630959,...,1931949.625,38250.0,7.485615,4615057.00,37428.074219,29942.458984,557.217224,474.807770,12379738,-5.115908e-12
4,23986.330980,1.0,37483,2600,8507.287887,113880040,0.093598,2346389.50,32317458,64.625946,...,1931949.625,38250.0,7.496615,4615474.50,37483.074219,29986.460938,557.270813,474.850708,12381512,-4.547474e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,27658.543066,NaN,39972,2600,17216.133207,117987800,0.096935,2404893.25,34391576,64.574020,...,NaN,NaN,7.994416,5329744.50,39972.078125,31977.664062,643.226257,548.336487,12404789,-1.485034e-11
1996,23523.250248,2.0,38210,2600,9065.010301,117988568,0.096944,2423196.25,33756112,65.625191,...,3863899.250,76500.0,7.642015,4150711.25,38210.078125,30568.060547,501.532043,427.034790,12035557,-1.259082e-11
1997,22723.580908,NaN,45458,2600,14321.051908,117993408,0.096978,2457797.50,32558184,63.991653,...,NaN,NaN,9.091619,5360958.50,45458.089844,36366.472656,647.312317,551.547852,12603720,-1.423928e-11
1998,22059.216738,2.0,39843,2600,7915.218891,117997224,0.096969,2439047.00,33213790,65.450035,...,3863899.250,76500.0,7.968616,4155376.50,39843.078125,31874.462891,502.191681,427.514771,12094723,-4.206413e-12


In [59]:
#pulling interesting data points from each LOC VAR Case
opt_out = pd.DataFrame(columns=['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/GenLarge\n(kW)', 'Architecture/1kWh LI',
                                                 'Cost/COE\n($)','GenMin'])
new_cases = pd.DataFrame(columns=['FileName','CaseNum','Case1 LCOE','Case5 LCOE','Architecture/GenLarge\n(kW)'])
tencases = ['Case10','Case11', 'Case12', 'Case13','Case14','Case15','Case16']
n = len(names)
display(n)
for i in range(n):
    name = names[i] # Grabs Sheet Name
    KCC = name[-3:]     #grabs KCC from last 3 values
    if KCC[-2:].casefold() == ('Af'.casefold()):
            KCC = 'Af'
            Location = name[5:-2]
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
            elif name[:6] == 'Case10':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
            else:
                Location = name[5:-2]
                case_num = name[:5]
    elif KCC[-2:].casefold() == 'Am'.casefold():
            KCC = 'Am'
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
            elif name[:6] == 'Case10':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
            else:
                Location = name[5:-2]
                case_num = name[:5]
    else: 
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                Location = name[6:-3]
                case_num = name[:6] # Grabs Case Number 
            #elif name[:6] == 'Case10' or name[:6]=='Case':
            elif name[:6] in tencases:
                Location = name[6:-3]
                case_num = name[:6] # Grabs Case Number 
            else:
                Location = name[5:-3]
                case_num = name[:5]
    KCC = KCC.lower()
    filtered_out = output_df[name].filter(['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/GenLarge\n(kW)', 'Architecture/1kWh LI',
                                                 'Cost/COE\n($)']) #Filtering objects of interest
    filtered_out['CaseName'] = Location #adding actual case name to series
    filtered_out['CaseNum'] = case_num # adding case num to series
    filtered_out['KCC'] = KCC
    framed = filtered_out.iloc[[0]] #making the actual df
    framed.set_index(['CaseNum','CaseName'], inplace=True, drop=True) #building df w/ casename and num
    opt_out = pd.concat([framed,opt_out]) #concating all DFs together
  

    
# Data Cleanup    
opt_out = opt_out[ ['KCC'] + [ col for col in opt_out.columns if col != 'KCC' ] ] #sorting df to have KCC first
opt_out = opt_out.sort_index()
opt_out.to_excel(trim_directory +'/'+ "LocationalVarianceTrimmed.xlsx")


display(opt_out)
#display(new_cases)  

639

KCC  Architecture/PV\n(kW)  Architecture/G1500  \
CaseNum CaseName                                                     
Case1   Anchorage   dfc           67432.441304                28.0   
        Ashland     dfb           39799.095877                19.0   
        AtlantaGA   cfa           92802.980815                 3.0   
        BangorME    dfb           62211.324125                20.0   
        BendOre     bsk           40050.377085                23.0   
...                 ...                    ...                 ...   
Case8   RaleighNC   cfa           32126.631249                 7.0   
        ReddingCA   csa           26843.735929                 4.0   
        RichmondVA  cfa           37944.276532                 5.0   
        Sacramento  csa           19589.237011                 2.0   
        Syracuse    dfb           74305.687555                13.0   

                   Architecture/GenLarge\n(kW) Architecture/1kWh LI  \
CaseNum CaseName                                                      
Case1   Anchorage                          NaN               238148   
        Ashland                            NaN                83635   
        AtlantaGA                          NaN               121958   
        BangorME                           NaN               143357   
        BendOre                            NaN                93120   
...                                        ...                  ...   
Case8   RaleighNC                       3000.0                40384   
        ReddingCA                         3000                38229   
        RichmondVA                        3000                46024   
        Sacramento                      3000.0                32783   
        Syracuse                           NaN                77203   

                    Cost/COE\n($) GenMin  
CaseNum CaseName                          
Case1   Anchorage        0.437239    NaN  
        Ashland          0.207991    NaN  
        AtlantaGA        0.252627    NaN  
        BangorME         0.301640    NaN  
        BendOre          0.233319    NaN  
...                           ...    ...  
Case8   RaleighNC        0.236074    NaN  
        ReddingCA        0.233377    NaN  
        RichmondVA       0.244868    NaN  
        Sacramento       0.216778    NaN  
        Syracuse         0.220980    NaN  

[639 rows x 7 columns]

In [60]:
#pulling interesting data points from each MLR  Case
mlr_opt_out = pd.DataFrame(columns=['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/GenLarge\n(kW)', 'Architecture/1kWh LI',
                                                 'Cost/COE\n($)','GenMin'])
mlr_new_cases = pd.DataFrame(columns=['FileName','CaseNum','Case1 LCOE','Case5 LCOE','Architecture/GenLarge\n(kW)'])
tencases = ['Case10','Case11', 'Case12', 'Case13','Case14','Case15','Case16']
n_mlr = len(mlr_names)
for i in range(n_mlr):
    name = mlr_names[i] # Grabs Sheet Name
    KCC = name[-3:]     #grabs KCC from last 3 values
    if KCC[-2:].casefold() == ('Af'.casefold()):
            KCC = 'Af'
            Location = name[5:-2]
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
                mlr      = name[8:10]
            elif name[:6] == 'Case10':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
                mlr      = name[8:10]
            else:
                Location = name[5:-2]
                case_num = name[:5]
                mlr      = name[8:10]
    elif KCC[-2:].casefold() == 'Am'.casefold():
            KCC = 'Am'
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
                mlr      = name[8:10]
            elif name[:6] == 'Case10':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
                mlr      = name[8:10]
            else:
                Location = name[5:-2]
                case_num = name[:5]
                mlr      = name[8:10]
    else: 
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                Location = name[6:-3]
                case_num = name[:6] # Grabs Case Number 
                mlr      = name[8:10]
            #elif name[:6] == 'Case10' or name[:6]=='Case':
            elif name[:6] in tencases:
                Location = name[6:-3]
                case_num = name[:6] # Grabs Case Number 
                mlr      = name[8:10]
            else:
                Location = name[10:-3]
                case_num = name[:5]
                mlr      = name[8:10]
                #print(mlr)
    if "AZYuma".casefold() in Location.casefold():
        print(Location)
        Location = Location.replace("AZYuma","YumaAZ")
        print(Location)
    KCC = KCC.lower()
    # print(name)
    mlr_filtered_out = mlr_output_df[name].filter(['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/GenLarge\n(kW)', 'Architecture/1kWh LI', 'Cost/COE\n($)']) #Filtering objects of interest
    mlr_filtered_out['CaseName'] = Location #adding actual case name to series
    mlr_filtered_out['CaseNum'] = case_num # adding case num to series
    mlr_filtered_out['KCC'] = KCC
    mlr_filtered_out['mlr'] = mlr
    mlr_framed = mlr_filtered_out.iloc[[0]] #making the actual df
    mlr_framed.set_index(['CaseNum','CaseName','mlr'], inplace=True, drop=True) #building df w/ casename and num
    mlr_opt_out = pd.concat([mlr_framed,mlr_opt_out]) #concating all DFs together
  

    
# Data Cleanup    
mlr_opt_out = mlr_opt_out[ ['KCC'] + [ col for col in opt_out.columns if col != 'KCC' ] ] #sorting df to have KCC first
mlr_opt_out = mlr_opt_out.sort_index()
mlr_opt_out.to_csv(trim_directory +'/'+ "MLRTrimmed.csv")


display(mlr_opt_out)
#display(new_cases)  

AZYuma
YumaAZ
AZYuma
YumaAZ


KCC  Architecture/PV\n(kW)  Architecture/G1500  \
CaseNum CaseName  mlr                                                   
Case2   Anchorage 50   dfc            6900.071875                 NaN   
                  60   dfc            7111.227366                 NaN   
                  70   dfc            7123.984923                 NaN   
                  80   dfc            7123.984923                 NaN   
                  90   dfc            7123.984923                 NaN   
...                    ...                    ...                 ...   
Case4   YumaAZ    60   bwh           18946.593167                 2.0   
                  70   bwh           18946.593167                 2.0   
                  80   bwh           18946.593167                 2.0   
                  90   bwh           18946.593167                 2.0   
                  95   bwh           18946.593167                 2.0   

                      Architecture/GenLarge\n(kW)  Architecture/1kWh LI  \
CaseNum CaseName  mlr                                                     
Case2   Anchorage 50                         5000                1500.0   
                  60                         5000                1655.0   
                  70                         5000                1606.0   
                  80                         5000                1606.0   
                  90                         5000                1606.0   
...                                           ...                   ...   
Case4   YumaAZ    60                       3000.0               34290.0   
                  70                       3000.0               34290.0   
                  80                       3000.0               34290.0   
                  90                       3000.0               34290.0   
                  95                       3000.0               34290.0   

                       Cost/COE\n($) GenMin  
CaseNum CaseName  mlr                        
Case2   Anchorage 50        0.071879    NaN  
                  60        0.071792    NaN  
                  70        0.071756    NaN  
                  80        0.071734    NaN  
                  90        0.071719    NaN  
...                              ...    ...  
Case4   YumaAZ    60        0.159500    NaN  
                  70        0.159500    NaN  
                  80        0.159500    NaN  
                  90        0.159500    NaN  
                  95        0.159500    NaN  

[1116 rows x 7 columns]

In [61]:
#fixing Utility File Names
os.chdir('d:/repo/homer/ClimateStudies/')
homer_util_dirs = os.path.abspath(os.getcwd()) + '/' + 'ProductionCases/UtilityConnection'
#print(homer_util_dirs)

util_opt_out = pd.DataFrame(columns=['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/GenLarge\n(kW)', 'Architecture/1kWh LI',
                                                 'Cost/COE\n($)','GenMin'])
util_new_cases = pd.DataFrame(columns=['FileName','CaseNum','Case1 LCOE','Case5 LCOE','Architecture/GenLarge\n(kW)'])
n_util = len(util_names)

fixed_util_names = []
for path, subdirs, files in os.walk(homer_util_dirs):
    for name in files:
        if name[-6:] == '.homer':
            #print(name.replace('.homer',''))
            fixed_util_names.append(name.replace('.homer',""))
            
#print(fixed_util_names)

        
for i in util_names:
    index = util_names.index(i)
    match = [match for match in fixed_util_names if i in match]
    util_names[index] = match
    
print(util_names)

[['Utility1dot3MWe12CentsAnchorageDfc'], ['Utility1dot3MWe12CentsAshlandDfb'], ['Utility1dot3MWe12CentsAtlantaGACfa'], ['Utility1dot3MWe12CentsBangorMEDfb'], ['Utility1dot3MWe12CentsBendOreBsk'], ['Utility1dot3MWe12CentsBentonCountyOregonCsb'], ['Utility1dot3MWe12CentsBirminghamALCfa'], ['Utility1dot3MWe12CentsBismarckNDDfb'], ['Utility1dot3MWe12CentsBlaineCountyMontanaBSk'], ['Utility1dot3MWe12CentsBoiseIDBSk'], ['Utility1dot3MWe12CentsBrewsterMADfb'], ['Utility1dot3MWe12CentsBurlingtonVTDfa'], ['Utility1dot3MWe12CentsButtonwillowBwk'], ['Utility1dot3MWe12CentsCarsonCityNVBsk'], ['Utility1dot3MWe12CentsCheyenneWYBsk'], ['Utility1dot3MWe12CentsChicagoDfa'], ['Utility1dot3MWe12CentsCoalvilleUTDfb'], ['Utility1dot3MWe12CentsColumbiaMOCfa'], ['Utility1dot3MWe12CentsColumbusOHDfa'], ['Utility1dot3MWe12CentsCrestedButteCODfc'], ['Utility1dot3MWe12CentsDesMoinesIADfa'], ['Utility1dot3MWe12CentsDetroitMIDfa'], ['Utility1dot3MWe12CentsEielsonAFBDfC'], ['Utility1dot3MWe12CentsEvergladesAM'], ['

In [64]:
#pulling interesting data points from each util  Case    
#os.chdir('OutputProcessing')
for i in range(n_util):
    name = util_names[i] # Grabs Sheet Name
    print(name)
    name = ''.join(name)
    if "6cents".casefold() in name.casefold():
        case_info = name[:21]
        price = name[15:16]
        if 'Utility2dot6MWe'.casefold() in case_info.casefold() :
            utility_str = 'Utility2dot6MWe' # used for string cleanup later
            util = '2.6MWe'
        elif'Utility1dot3MWe'.casefold() in case_info.casefold() :
            utility_str = 'Utility1dot3MWe' # used for string cleanup later
            util = '1.3MWe'
        else:
            print('error')
    else:     
        case_info = name[:22]
        price = name[15:17]
        if 'Utility2dot6MWe'.casefold() in case_info.casefold() :
            utility_str = 'Utility2dot6MWe' # used for string cleanup later
            util = '2.6MWe'
        elif'Utility1dot3MWe'.casefold() in case_info.casefold() :
            utility_str = 'Utility1dot3MWe' # used for string cleanup later
            util = '1.3MWe'
        else:
            print('error')
    KCC = name[-3:]     #grabs KCC from last 3 values
    if KCC[-2:].casefold() == ('Af'.casefold()):
            KCC = 'Af'
            Location = name[5:-2]
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
            elif name[:6] == 'Case10':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
            else:
                Location = name[5:-2]
                case_num = name[:5]
    elif KCC[-2:].casefold() == 'Am'.casefold():
            KCC = 'Am'
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
            elif name[:6] == 'Case10':
                Location = name[6:-2]
                case_num = name[:6] # Grabs Case Number 
            else:
                Location = name[5:-2]
                case_num = name[:5]
    else: 
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                Location = name[6:-3]
                case_num = name[:6] # Grabs Case Number 
            #elif name[:6] == 'Case10' or name[:6]=='Case':
            elif name[:6] in tencases:
                Location = name[6:-3]
                case_num = name[:6] # Grabs Case Number 
            else:
                Location = name.replace(utility_str, "")
                Location = Location.replace("Cents", "")
                Location = Location.replace(KCC, "")
                Location = Location.replace(str(12), "")
                case_num = name[:5]
    KCC = KCC.lower()
    #print(name)
    util_filtered_out = util_output_df[name].filter(['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/GenLarge\n(kW)', 'Architecture/1kWh LI', 'Cost/COE\n($)']) #Filtering objects of interest
    util_filtered_out['CaseName'] = Location #adding actual case name to series
    util_filtered_out['CaseNum'] = case_num # adding case num to series
    util_filtered_out['KCC'] = KCC
    #util_filtered_out['util'] = util add back to .set_index line
    util_framed = util_filtered_out.iloc[[0]] #making the actual df
    util_framed.set_index(['CaseNum','CaseName'], inplace=True, drop=True) #building df w/ casename and num
    util_opt_out = pd.concat([util_framed,util_opt_out]) #concating all DFs together
  

    
# Data Cleanup    
util_opt_out = util_opt_out[ ['KCC'] + [ col for col in opt_out.columns if col != 'KCC' ] ] #sorting df to have KCC first
util_opt_out = util_opt_out.sort_index()
util_opt_out.to_csv(trim_directory +'/'+"utilTrimmed.csv")


display(util_opt_out)
#display(new_cases)  

['Utility1dot3MWe12CentsAnchorageDfc']


KeyError: 'Utility1dot3MWe12CentsAnchorageDfc'

In [ ]:
# DATA FOR LOC VAR
#for index = opt_out.ind 
#display(pd.MultiIndex.get_level_values(opt_out('CaseNum')))
CFA = opt_out.query("KCC=='cfa'")
print(CFA.query("CaseNum=='Case1'"))
CFA.plot(x='CaseName',y='Cost/COE\n($)')
Case1 = opt_out.loc[("Case1",slice(None))]
Case1CFA = Case1.query("KCC=='cfa'")
Case2 = opt_out.loc[("Case2",slice(None))]
Case2CFA = Case2.query("KCC=='cfa'")
Case25 = opt_out.loc[("Case25",slice(None))]
Case3 = opt_out.loc[("Case3",slice(None))]
Case3CFA = Case3.query("KCC=='cfa'")
Case35 = opt_out.loc[("Case35",slice(None))]
Case4 = opt_out.loc[("Case4",slice(None))]
Case4CFA = Case4.query("KCC=='cfa'")
display(Case4CFA)
# Testing
Case5 = opt_out.loc[("Case5",slice(None))]
Case6 = opt_out.loc[("Case6",slice(None))]
Case7 = opt_out.loc[("Case7",slice(None))]
Case8 = opt_out.loc[("Case8",slice(None))]
Case10 = opt_out.loc[("Case10",slice(None))]
Case11 = opt_out.loc[("Case11",slice(None))]
Case12 = opt_out.loc[("Case12",slice(None))]
Case13 = opt_out.loc[("Case13",slice(None))]
Case14 = opt_out.loc[("Case14",slice(None))]
Case15 = opt_out.loc[("Case15",slice(None))]
Case16 = opt_out.loc[("Case16",slice(None))]
display(Case16)
Case1.plot(y='Cost/COE\n($)', use_index=True)


In [ ]:
# DATA FOR MLR
#for index = opt_out.ind 
#display(pd.MultiIndex.get_level_values(opt_out('CaseNum')))
testcase = "Anchorage"
Case2= mlr_opt_out.loc[("Case2",slice(None))]
Case3= mlr_opt_out.loc[("Case3",slice(None))]
Case4= mlr_opt_out.loc[("Case4",slice(None))]
Case2test = Case2.loc[(testcase,slice(None))]
Case3test = Case3.loc[(testcase,slice(None))]
Case4test = Case4.loc[(testcase,slice(None))]
fig = plt.figure()
fig, ax=plt.subplots(figsize=(8,4))
# 'Architecture/GenLarge\n(kW)'
ax.scatter( Case2test.index.array, Case2test['Architecture/GenLarge\n(kW)'],marker='*',s=200)
ax.scatter( Case3test.index.array, Case3test['Architecture/GenLarge\n(kW)'],marker='*',s=200)
ax.scatter( Case4test.index.array, Case4test['Architecture/GenLarge\n(kW)'],marker='*',s=200)
ax.legend(['$8,000/kWe','$16,000/kWe','$24,000/kWe'], fontsize='large')
plt.ylabel('LCOE [$/MWhre]')
plt.xlabel('minimum load ratio [%]')
plt.title('Levelized Cost of Electricity for All Studied Locations w/Assumed MR Capital Cost', fontsize='large')
plt.xticks(rotation=90);
ax.grid(True)

plt.savefig("mlr_testing")

In [ ]:
# DATA FOR util
#for index = opt_out.ind 
#display(pd.MultiIndex.get_level_values(opt_out('CaseNum')))
Case1 = util_opt_out.loc[("Case1",slice(None))]


In [ ]:
with plt.rc_context():
    plt.rcParams['axes.prop_cycle'] = cycler(color=['black', 'red', 'orange', 'saddlebrown', 'gold','olivedrab','lawngreen',
                                                    'darkgreen','lime','lightseagreen','cyan','deepskyblue','blue',
                                                    'indigo', 'fuchsia','crimson', 'lightpink'])
    fig = plt.figure()
    fig, ax=plt.subplots(figsize=(36,12))
    ax.scatter( Case1.index.array, Case1['Cost/COE\n($)']*1000,marker='*',s=200)
    ax.scatter( Case1.index.array, Case2['Cost/COE\n($)']*1000)
    ax.scatter( Case1.index.array,Case25['Cost/COE\n($)']*1000)
    ax.scatter( Case1.index.array, Case3['Cost/COE\n($)']*1000)
    ax.scatter( Case1.index.array,Case35['Cost/COE\n($)']*1000)
    ax.scatter( Case1.index.array, Case4['Cost/COE\n($)']*1000)
    ax.scatter( Case5.index.array, Case5['Cost/COE\n($)']*1000)
    ax.scatter( Case6.index.array, Case6['Cost/COE\n($)']*1000)
    ax.scatter( Case7.index.array, Case7['Cost/COE\n($)']*1000)
    ax.scatter( Case8.index.array, Case8['Cost/COE\n($)']*1000)
    ax.scatter(Case10.index.array,Case10['Cost/COE\n($)']*1000)
    ax.scatter(Case11.index.array,Case11['Cost/COE\n($)']*1000)
    ax.scatter(Case12.index.array,Case12['Cost/COE\n($)']*1000)
    ax.scatter(Case13.index.array,Case13['Cost/COE\n($)']*1000)
    ax.scatter(Case14.index.array,Case14['Cost/COE\n($)']*1000)
    ax.scatter(Case15.index.array,Case15['Cost/COE\n($)']*1000)
    ax.scatter(Case15.index.array,Case15['Cost/COE\n($)']*1000)
    ax.legend(['Only Renewables', '$8,000/kWe','$12,000/kWe', 
               '$16,000/kWe', '$20,000/kWe','$24,000/kWe', '$30,000/kWe','$35,000/kWe','$40,000/kWe',
               '$45,000/kWe','$55,000/kWe','$60,000/kWe','$65,000/kWe','$70,000/kWe','$80,000/kWe',
               '$85,000/kWe','$95,000/kWe'], fontsize='large')
    plt.ylabel('LCOE [$/MWhre]')
    plt.xlabel('Location')
    plt.title('Levelized Cost of Electricity for All Studied Locations w/Assumed MR Capital Cost', fontsize='large')
    plt.xticks(rotation=90);
    ax.grid(True)


In [ ]:
m = len(Case2CFA)
fig = plt.figure(figsize=(24,12))
barWidth = 0.25
# Grabbing Generator Size Data
bars1 = Case2CFA["Architecture/GenLarge\n(kW)"]
#bars15 = Case25CFA["Architecture/GenLarge\n(kW)"]
bars2 = Case3CFA["Architecture/GenLarge\n(kW)"]
#bars25 = Case35CFA["Architecture/GenLarge\n(kW)"]
bars3 = Case4CFA["Architecture/GenLarge\n(kW)"]
#ranges
r1 = np.arange(m)
#r15 = [x+barWidth for x in r1]
r2 = [x+barWidth for x in r1]
#r25 = [x+barWidth for x in r2]
r3 = [x+barWidth for x in r2]

# Plotting
plt.bar(r1, bars1, width=barWidth, edgecolor='white', label='LowCostNuclear')
#plt.bar(r15, bars15, width=barWidth, edgecolor='white', label='LoMidCostNuclear')
plt.bar(r2, bars2, width=barWidth, edgecolor='white', label='MidCostNuclear')
#plt.bar(r25, bars25, width=barWidth, edgecolor='white', label='MidCostNuclear')
plt.bar(r3, bars3, width=barWidth, edgecolor='white', label='HighCostNuclear')
# Add xticks on the middle of the group bars
plt.ylabel('Installed Nuclear Capacity [MWe]',fontweight='bold')
plt.xlabel('[Cfa] Humid Subtropical Locations',fontweight='bold')
plt.title('Predicted Installed Nuclear Capacity for Modeled Cfa Locations',fontweight='bold')
plt.xticks([r + barWidth for r in range(m)], Case2CFA.index.values)
plt.xticks(rotation=90);
plt.grid(True)
 


In [ ]:
# Initial KCC Plots
with plt.rc_context():
    plt.rcParams['axes.prop_cycle'] = cycler(color=['black', 'red', 'orange', 'saddlebrown', 'gold','olivedrab','lawngreen',
                                                    'darkgreen','lime','lightseagreen','cyan','deepskyblue','blue',
                                                    'indigo', 'fuchsia','crimson'])
    fig = plt.figure()
    fig, ax=plt.subplots(figsize=(12,6))
    #ax.scatter( Case1CFA.index.array, Case1CFA['Architecture/GenLarge\n(kW)']*1000,marker='*',s=200)
    ax.bar( Case2CFA.index.array, Case2CFA['Architecture/GenLarge\n(kW)']/1000)
    ax.bar( Case3CFA.index.array, Case3CFA['Architecture/GenLarge\n(kW)']/1000)
    ax.bar( Case4CFA.index.array, Case4CFA['Architecture/GenLarge\n(kW)']/1000)
    ax.legend([ 'LowCostNuclear', 'MidCostNuclear', 'HighCostNuclear'], fontsize='large')
    plt.ylabel('Installed Nuclear Capacity [MWe]')
    plt.xlabel('[Cfa] Humid Subtropical Locations')
    plt.xticks(rotation=90);
    #plt.yticks([3,4,5])
    ax.grid(True)